In [1]:
import random
from faker import Faker
import pandas as pd

In [13]:
fake = Faker('en_US')

In [34]:
fake.company_suffix()

'Inc'

In [38]:
# Generate Customers
def generate_customers(num_customers=100):
    customers = []
    for _ in range(num_customers):
        customer = {
            'CustomerID': fake.uuid4(),
            'Name': fake.company(),
            'CompanyType':fake.company_suffix(),
            'ContactDetails': fake.phone_number(),
            'EmailDetails': fake.company_email(),
            'AddressDetails':fake.address(),
            'AdminDetails':fake.administrative_unit(),
            'CreditLimit': random.randint(10000, 100000),
            'CreditRating': random.choice(['very poor', 'fair', 'good', 'very good','exceptional'])
        }
        customers.append(customer)
    return customers

In [39]:
# Generate Data
customers = generate_customers()

In [41]:
df_customers = pd.DataFrame(customers)

In [42]:
df_customers

,CustomerID,Name,CompanyType,ContactDetails,EmailDetails,AddressDetails,AdminDetails,CreditLimit,CreditRating
0,e64432be-37c5-4c62-8672-672c0a099bde,Arroyo-Gutierrez,Ltd,474.564.4195x9582,armstrongjessica@cisneros.biz,"596 Kevin Burgs\nNew Matthew, PA 84375",Texas,65846,very good
1,844a6039-784e-4c8d-8edb-c48b66bc4541,Miller and Sons,LLC,419-315-6151,leachtodd@taylor.net,USNS Smith\nFPO AE 17368,Tennessee,44399,exceptional
2,e9940061-d1cc-4ed9-8a3b-9b823ebdc454,"Long, Thompson and Swanson",PLC,001-141-104-0113,ldillon@harvey-thomas.biz,"60030 Jeffery Springs\nNunezside, WA 69622",North Dakota,74716,fair
3,298d88f1-bff5-4025-a185-6384bb838de4,Jackson-Roberts,Inc,(973)582-4055x12514,josepholson@wilson-ramirez.com,"4158 Taylor Springs\nHortonview, OK 07511",South Dakota,50467,very poor
4,50f80e48-bd60-4401-a03c-9997dd038acb,"Williams, Summers and Murphy",Inc,(489)565-9177x06381,nbrown@hughes-ruiz.com,"539 Dominic Mills Apt. 135\nPort Tammyburgh, O...",Louisiana,12423,good
...,...,...,...,...,...,...,...,...,...
95,e0af290e-d051-47a7-9119-446e49f20e6d,Anderson PLC,Inc,635.248.7310x4896,macdonaldrobert@rodriguez.com,"88989 Virginia Fords\nDoristown, MT 30847",Connecticut,36213,very good
96,df20e19d-248a-4aa4-ba4a-66b818c5d426,"Maxwell, Cox and Novak",LLC,6016595326,cnicholson@rodriguez.biz,"22461 Stone Lock Apt. 913\nParrishstad, VT 24000",Iowa,96411,fair
97,b4144210-def9-48d4-8cdd-eb782e779ea0,"Lowe, Bautista and Houston",Inc,246-643-1541x51918,dfuller@cochran-lutz.com,"627 Susan Mountain\nWest Gregoryview, DC 04853",Kansas,98655,exceptional
98,b980b394-3512-44b2-ae81-ce735b764ee7,Thomas Group,PLC,(796)296-1267x921,michaelarnold@walker.com,"PSC 7320, Box 0070\nAPO AP 35331",Alabama,68963,very poor


In [44]:
# Define the product types for each category
product_types = {
    'Office Supplies': [
        "Laser Printer",
        "Ergonomic Chair",
        "Standing Desk",
        "Wireless Keyboard",
        "Office Stationery Set"
    ],
    'Healthcare': [
        "Digital Thermometer",
        "Blood Pressure Monitor",
        "Glucometer",
        "Surgical Masks",
        "Hand Sanitizer"
    ],
    'Consumer Goods': [
        "LED Light Bulbs",
        "Vacuum Cleaner",
        "Air Purifier",
        "Electric Kettle",
        "Microwave Oven"
    ],
    'Automotive': [
        "Car Battery",
        "Engine Oil",
        "Brake Pads",
        "Tires",
        "Car Air Freshener"
    ],
    'Construction Materials': [
        "Cement",
        "Steel Beams",
        "Insulation Material",
        "PVC Pipes",
        "Roofing Sheets"
    ],
    'Food Beverages': [
        "Organic Coffee Beans",
        "Bottled Water",
        "Energy Drink",
        "Protein Bars",
        "Gourmet Chocolate"
    ],
    'Industrial Equipment': [
        "Forklift",
        "Conveyor Belt",
        "Industrial Robot",
        "3D Printer",
        "CNC Machine"
    ],
    'Electronics': [
        "Laptop",
        "Smartphone",
        "Tablet",
        "Smartwatch",
        "Bluetooth Headphones"
    ]
}

In [103]:
products = {'Laser Printer': [100,425],
'Ergonomic Chair': [1000,299],
'Standing Desk': [1000,420],
'Digital Thermometer': [2500,36],
'Blood Pressure Monitor': [2500,44],
'Glucometer': [2500,29],
'Vacuum Cleaner': [100,364],
'Electric Kettle': [500,199],
'Microwave Oven': [500,325],
'Car Battery': [1000,99],
'Engine Oil': [1000,40],
'Brake Pads': [1000,60],
'Tires': [2000,120],
'Car Air Freshener': [2500,9],
'Cement': [500,100],
'Forklift': [50,21000],
'Conveyor Belt': [1000,1000],
'Industrial Robot': [50,30000],
'3D Printer': [100,980],
'CNC Machine': [100,25000],
'Wireless Keyboard': [1000,78],
'LED Light Bulbs': [5000,5],
'Air Purifier': [1000,99],
'Steel Beams': [1000,600],
'Insulation Material': [10000,2],
'PVC Pipes': [5000,3],
'Roofing Sheets': [5000,7],
'Laptop': [1000,799],
'Smartphone': [1000,675],
'Tablet': [1000,499],
'Smartwatch': [1000,160],
'Bluetooth Headphones': [1000,139],
'Organic Coffee Beans': [5000, 19],
'Bottled Water': [10000, 5],
'Energy Drink': [5000, 7],
'Protein Bars': [5000, 5],
'Gourmet Chocolate': [5000, 9],
'Office Stationery Set':[1000, 15],
'Surgical Masks' : [10000,2]
}

In [45]:
# Function to assign product type based on product category
def assign_product_type(product_category):
    if product_category in product_types:
        return random.choice(product_types[product_category])
    else:
        return None

# Example usage
product_categories = [
    'Office Supplies', 'Healthcare', 'Consumer Goods', 'Automotive',
    'Construction Materials', 'Food Beverages', 'Industrial Equipment', 'Electronics'
]

In [107]:
def assign_order_quantity_and_price(sample_product_type):
    order_details = {}
    for product, values in products.items():
        if product == sample_product_type:
            max_order_limit, unit_price = values
            order_quantity = random.randint(max_order_limit-50, max_order_limit)
            order_price = order_quantity * unit_price
    return order_quantity, order_price, unit_price

In [111]:
# Generate Products
def generate_products(num_products=30):
    products = []
    for _ in range(num_products):
        sample_product_categories = random.choice(product_categories)
        sample_product_type = assign_product_type(sample_product_categories)
        print (sample_product_categories, sample_product_type)
        sample_product_quantity, sample_product_price, stock_price = assign_order_quantity_and_price(sample_product_type)
        product = {
            'ProductID': fake.uuid4(),
            # Create a sample list of product categories
            'ProductCategory' : sample_product_categories,
            'ProductType' : sample_product_type,            
            'Price': sample_product_price,
            'Quantity': sample_product_quantity,
            'UnitPrice' : stock_price
        }
        products.append(product)
    return products

In [112]:
products = generate_products()

Healthcare Blood Pressure Monitor


AttributeError: 'list' object has no attribute 'items'

In [105]:
df_products = pd.DataFrame(products)

In [106]:
df_products

,ProductID,ProductCategory,ProductType,Price,Quantity
0,4644ab0d-367c-4374-b814-f2cc8e99b97d,Construction Materials,PVC Pipes,14901,4967
1,8710d85f-4ce5-48d9-99c7-3c8ad7328dd0,Electronics,Bluetooth Headphones,137471,989
2,0f74cd0a-d552-49a3-9db5-96f930171503,Industrial Equipment,CNC Machine,1300000,52
3,4a2bbfdd-49ea-40cc-832e-77cd9dd1ec49,Office Supplies,Standing Desk,412440,982
4,152f9c9e-df73-4b66-afaf-d672297bc77e,Construction Materials,Steel Beams,586200,977
5,1b1a19d8-d34f-45be-87d5-3eaa9cf05b42,Automotive,Tires,238080,1984
6,2c385ee2-688a-4a1f-adb5-c519a447c22c,Food Beverages,Energy Drink,34755,4965
7,dc3b20e6-862e-4ab3-9b2c-33db9bd23e56,Industrial Equipment,3D Printer,53900,55
8,933afefe-6571-4534-b9d6-5c736b56f954,Food Beverages,Protein Bars,24945,4989
9,1338c115-7e52-482b-85c1-993b96cadef4,Food Beverages,Gourmet Chocolate,44586,4954


In [ ]:
# Generate Orders and Order Items
def generate_orders_and_items(customers, products, num_orders=100000):
    orders = []
    order_items = []
    shipments = []
    invoices = []
    payments = []
    statuses = ['Fulfilled', 'Delayed Fulfillment', 'Pending Payment']

    for _ in range(num_orders):
        customer = random.choice(customers)
        order_status = random.choice(statuses)
        order_id = fake.uuid4()
        order_date = fake.date_this_year()
        total_amount = 0

        order = {
            'OrderID': order_id,
            'CustomerID': customer['CustomerID'],
            'OrderDate': order_date,
            'OrderStatus': order_status,
            'TotalAmount': total_amount  # Will be updated later
        }
        orders.append(order)

        num_items = random.randint(1, 5)
        for _ in range(num_items):
            product = random.choice(products)
            quantity = random.randint(1, 10)
            total_price = product['Price'] * quantity
            total_amount += total_price

            order_item = {
                'OrderItemID': fake.uuid4(),
                'OrderID': order_id,
                'ProductID': product['ProductID'],
                'Quantity': quantity,
                'UnitPrice': product['Price'],
                'TotalPrice': total_price
            }
            order_items.append(order_item)

        # Update the total amount for the order
        order['TotalAmount'] = total_amount

        shipment = {
            'ShipmentID': fake.uuid4(),
            'OrderID': order_id,
            'ShipmentDate': fake.date_this_year(),
            'Carrier': fake.company(),
            'TrackingNumber': fake.uuid4(),
            'ShipmentStatus': 'Shipped' if order_status == 'Fulfilled' else 'Pending'
        }
        shipments.append(shipment)

        invoice = {
            'InvoiceID': fake.uuid4(),
            'OrderID': order_id,
            'InvoiceDate': fake.date_this_year(),
            'DueDate': fake.date_this_year(),
            'TotalAmount': total_amount,
            'PaymentStatus': 'Paid' if order_status == 'Fulfilled' else 'Pending'
        }
        invoices.append(invoice)

        if order_status == 'Fulfilled':
            payment = {
                'PaymentID': fake.uuid4(),
                'InvoiceID': invoice['InvoiceID'],
                'PaymentDate': fake.date_this_year(),
                'PaymentAmount': total_amount,
                'PaymentMethod': random.choice(['Credit Card', 'Bank Transfer', 'Cash'])
            }
            payments.append(payment)

    return orders, order_items, shipments, invoices, payments

In [ ]:
orders, order_items, shipments, invoices, payments = generate_orders_and_items(customers, products)

In [ ]:
# Convert to DataFrames for easy manipulation


df_orders = pd.DataFrame(orders)
df_order_items = pd.DataFrame(order_items)
df_shipments = pd.DataFrame(shipments)
df_invoices = pd.DataFrame(invoices)
df_payments = pd.DataFrame(payments)